## SMS 보내기(카페24)

카페24에서는 PHP/ASP/JSP 샘플만 제공된다. 이를 파이썬 코드로 porting해서 사용해야함

## 문자열 바이트 수 
- 유니코드 문자열에서는 글자수를 카운트한다.
- 특정 인코딩 문자열에서는 바이트 수 (ex)euckr, utf-8

In [1]:
print(len('한글'))

In [2]:
# len('<문자열>'. encode('형식'))
print(len('한글'.encode('euckr')))

4


In [3]:
print(len('한글'.encode('utf8')))

6


### 특정 인코딩의 문자열 자르기
#### 인코딩과 디코딩 하기

In [4]:
sample = '안녕하세요. 좋은 아침입니다'

result = sample.encode('euckr')
print(result)

b'\xbe\xc8\xb3\xe7\xc7\xcf\xbc\xbc\xbf\xe4. \xc1\xc1\xc0\xba \xbe\xc6\xc4\xa7\xc0\xd4\xb4\xcf\xb4\xd9'


In [5]:
# 인코딩을 euckr로 했다면 디코딩도 euckr로 해야 에러가 발생하지 않는다.
result.decode('euckr')

'안녕하세요. 좋은 아침입니다'

#### ignore옵션을 사용하면 str을 슬라이싱해도 에러가 발생하지 않는다.

In [6]:
result = sample.encode('euckr')[8:]
result.decode('utf-8', 'ignore').encode('utf-8')

b'.  \xc4\xa7\xd4\xb4\xcf\xb4'

## SMS 잔여건수 조회하기
### php코드
```
$sms_url = "http://sslsms.cafe24.com/sms_remain.php"; // 전송요청 URL
$sms['user_id'] = base64_encode("<SMS 아이디>"); // SMS 아이디
$sms['secure'] = base64_encode("<secure Key>") ;//인증키   
$sms['mode'] = base64_encode("1"); // base64 사용시 반드시 모드값을 1로 주셔야 합니다.
```

In [7]:
from base64 import b64encode
import requests

In [8]:
sms_url = 'https://sslsms.cafe24.com/sms_remain.php'
user_id = b64encode('유저아이디'.encode('euckr')) # 아이디
secure = b64encode('시큐어키'.encode('euckr')) # 인증키
mode = b64encode('1'.encode('euckr')) # base64 사용시 반드시 모드값을 1로 주셔야 합니다.

In [9]:
response = requests.post(sms_url, data={
    'user_id': user_id,
    'secure': secure,
    'mode': mode,
})

In [10]:
response.text

'296'

## 문자 발송 요청

In [16]:
params = {
    'sms_url': sms_url,
    'user_id': user_id,
    'secure': secure,
    'mode': mode, 
    'sphone1': '010',
    'sphone2': '앞4자리',
    'sphone3': '뒤4자리',
    'rphone' : '수신인번호',
    'msg': '테스트'
}
print(params)
data = {}
for key, value in params.items():
    if isinstance(value, str):
        value = value.encode('euckr')
        if key == 'msg':
            value = value[:90].decode('euckr', 'ignore').encode('euckr')
        data[key] = value

{'sms_url': 'https://sslsms.cafe24.com/sms_remain.php', 'user_id': '<user_id>', 'secure': '<secure key>', 'mode': b'MQ==', 'sphone1': '010', 'sphone2': '앞4자리', 'sphone3': '뒤4자리', 'rphone': '수신인번호', 'msg': '테스트'}


In [17]:
response = requests.post("https://sslsms.cafe24.com/sms_sender.php", data=data)

In [18]:
response.text

'-102,'

## 코드 리펙토링, 여러개 발송

In [14]:
def send_sms(user_id, secure, sender, receivers, message):
    params = {
        'user_id': user_id,
        'secure': secure,
        'mode': '1',
        'sphone1': sender[:3],
        'sphone2': sender[3:-4],
        'sphone3': sender[-4:],
        'rphone': ','.join(receivers), # 받는 사람
        'msg': message,
    }

    data = {}
    for key, value in params.items():
        if isinstance(value, str):
            value = value.encode('euckr')
            if key == 'msg': # 90바이트까지 인코딩하여 발송하는 설정 부분
                value = value[:90].decode('euckr', 'ignore').encode('euckr')
            data[key] = value
            
    response = requests.post("http://sslsms.cafe24.com/sms_sender.php", data=data)
    response.text

In [15]:

user_id = '<user_id>'
secure = '<secure key>'
sender = '<발신번호>'
receiver = ['<수신번호>', '<수신번호>']
message = '<메시지>'

send_sms(user_id, secure, sender, receiver, message)